# Workshop Two - Hangman

As a way to recap what we've learnt previously, let's make a version of the popular word guessing game Hangman.

## Task 0: Understanding the Logic
Before we begin, let's break down how hangman works.

1. User guesses a letter.
2. Check if the letter has been guessed or not. If yes, return to 1.
3. Check if the letter is in the word.
    1. If true, reveal the letter in the word.
    2. Otherwise, increase the number of guesses used.
4. Add the letter to the guessed pile.
5. If the user guessed all the letters or ran out of guesses, end the game. Otherwise, go back to 1.

This is what we will be doing today, step by step.

## Task 1: Code Cleanup
The first thing we have to do is to write a function to print out the hangman figure. Luckily, someone has written out the code for us already below:

In [1]:
import time # for pausing the program
from IPython.display import clear_output # for clearning the output
import random # For generating random numbers and words
from urllib.request import urlopen # a surprise tool that will help us later! (note that this is somewhat different syntax than our usual imports)
import json # another surpise tool! We somehow have more tools than what Mickey usually has.

In [2]:
def to_suffix(i): # Helper function to generate st, nd, rd, etc. Should work only for 1 to 9
    if i == 1:
        return "st"
    elif i == 2:
        return "nd"
    elif i == 3:
        return "rd"
    else:
        return "th"

In [3]:
def draw_simple(count):
    if count == 0:
        print('\n\n\n\n\n\n\n_|_\n')
    elif count == 1:
        print(	'   _____ \n'
			'  |      \n'
			'  |      \n'
			'  |      \n'
			'  |      \n'
			'  |      \n'
			'  |      \n'
            '__|__    \n')

    elif count == 2:
        print(	'   _____  \n'
            '  |     | \n'
            '  |     | \n'
            '  |       \n'
            '  |       \n'
            '  |       \n'
            '  |       \n'
            '__|__     \n')

    elif count == 3:
        print(  '   _____  \n'
            '  |     | \n'
            '  |     | \n'
            '  |     | \n'
            '  |       \n'
            '  |       \n'
            '  |       \n'
            '__|__     \n')

    elif count == 4:
        print(	'   _____  \n'
            '  |     | \n'
            '  |     | \n'
            '  |     | \n'
            '  |     O \n'
            '  |       \n'
            '  |       \n'
            '__|__     \n')

    elif count == 5:
        print(	'   _____   \n'
            '  |     |  \n'
            '  |     |  \n'
            '  |     |  \n'
            '  |     O  \n'
            '  |    /|\ \n'
            '  |        \n'
            '__|__      \n')

    elif count == 6:
        print(	'   _____   \n'
            '  |     |  \n'
            '  |     |  \n'
            '  |     |  \n'
            '  |     O  \n'
            '  |    /|\ \n'
            '  |    / \ \n'
            '__|__      \n')
        print()

Let's see this in action.

In [4]:
for i in range(7): # from 0 to 6 (not to 7!) do..
    print(f"The {i}{to_suffix(i)} loop looks like this:")
    draw_simple(i) # use our function
    time.sleep(1) # pauses the program for one second
    clear_output(wait=True) # clears our output so we have an animation

The 6th loop looks like this:
   _____   
  |     |  
  |     |  
  |     |  
  |     O  
  |    /|\ 
  |    / \ 
__|__      




This definitely works, but the code seems clunky. Can we modify this to not repeat ourselves that much? 

Try to complete the following function without changing how the code works.

Replace all the `None`s with the appropriate conditional.

Here, `drawing_number` corresponds to which hangman shape to draw.

In [5]:
def draw(drawing_number):

    # First line of output (the ____)
    if drawing_number > 0:
        print('   _____ ') # If drawining number > 0, then...

        # there are 6 lines that need to be output each time, corresponding to each |
        # make a loop which runs 6 times here by replacing the [] in the for loop
        # hint: use range()
        for line_number in range(6):
            print('  |    ', end='') # we print the basic | on the left first, and with 4 spaces (we also use the end='' introduced last to to prevent an extra enter
    
            # if drawing number > 1 and the line number == 0 or == 1, then...
            if drawing_number > 1 and (line_number == 0 or line_number == 1):
                print(' |')
            elif drawing_number > 2 and line_number == 2: # if drawing number > 2 and the line number == 2, then...
                print(' |')
            elif drawing_number > 3 and line_number == 3: # if drawing number > 3 and the line number == 3, then...
                print(' O')
            elif drawing_number > 4 and line_number == 4: # if drawing number > 4 and the line number == 4, then...
                print('/|\\') # Note that we use \\ to represent 1 \ only in a string
            elif drawing_number > 5 and line_number == 5: # if drawing number > 5 and the line number == 5, then...
                print('/ \\')
            else: # we still need to input an enter to make it go to the next line
                print()

    print('__|__     \n') # the floor of the frame, needed every time

## Test: Does your code work?
Test if your output is identical to the one above by running this block.

In [6]:
for i in range(0, 7): # from 0 to 6 (not to 7!) do..
    print(f"The {i}{to_suffix(i)} loop looks like this:")
    draw(i) # use our function
    time.sleep(1) # pauses the program for one second
    clear_output(wait=True) # clears our output so we have an animation

The 6th loop looks like this:
   _____ 
  |     |
  |     |
  |     |
  |     O
  |    /|\
  |    / \
__|__     



## Generating Random Words
As we established last time, generating random stuff is *hard* even for programmers. This goes double for generating words that make sense.

Since Python does not have a built-in way to do this, we use our surprise tool (`urllib`) to grab a word list from the web.

The list is [here](https://github.com/bevacqua/correcthorse/blob/master/wordlist.json), and is actually inspired by [this XKCD](http://xkcd.com/936/).

Note: Please don't run this block (the one with `urlopen`) too much - you are using someone else's resources each time you run this.

In [7]:
words_json = urlopen("https://raw.githubusercontent.com/bevacqua/correcthorse/master/wordlist.json").read() # reads the word list
word_list = json.loads(words_json) # turns the words into a list

Let's generate a random word by using our good friend `random` again.

You can run this block however many times you want to get a new random word.

In [8]:
random_word = random.choice(word_list) # random.choice chooses something random from the word list
print(f"Your random word is: {random_word}")

Your random word is: member


## Task 2: Making Sure the Input is Valid
Before we begin, let's define all the variables we will use.

You can re-run this block to reset all the variables to their defaults.

In [9]:
guesses_used = 0 # The number of guesses used
limit = 6 # The maximum number of guesses before you lose
guessed = [] # The guessed number pile; a List
display = ['_'] * len(random_word) # The letters to display; Initially a List of all [_]
letters = list(random_word) # The actual letters of the word; will not be modified

First, we write a function to continuously prompt the user to input until they input something valid:

In [10]:
def get_guess(guessed):
    user_input = "" # initialize user_input to empty
    input_ok = False # set this to true if the input is something we want
    ## TODO HERE
    while not input_ok: # while the input is NOT ok (replace None with the correct answer)
        user_input = input("Please enter a letter: ") # ask user for input
        # this is a pretty complex conditonal, so it is given
        if (len(user_input) == 1) and user_input[0].isalpha(): # if the input length is 1, and the first letter is a character
            ## TODO HERE
            # if the user input has not been guessed already...
            # Replace the None here with the appropriate condition
            # Hint: How do you check if something is in the list?
            if user_input not in guessed:
                input_ok = True # Input is OK, so we make the loop exit
            else: # input has already been guessed
                print("The input has already been guessed! Please try again.")
        else: # input invalid, length != 1 or first letter not character
            print("The input is invalid. Please try again.")
    return user_input # give the user_input back

## Test: Does your code work?
Run the block below and test your code with different inputs!

(For the sake of testing, we set `a, e, i, o, u` to be guessed letters.)

In [11]:
guessed = ['a', 'e', 'i', 'o', 'u'] # Let's temporarily make [a, e, i, o, u] guessed letters...
print(f"The guess {get_guess(guessed)} is valid!")
guessed = [] # Reset the guessed list

Please enter a letter:  p


The guess p is valid!


## Task 3: Updating the Game State
Complete the function below to simulate what happens when a letter is guessed.

Please replace all the `None`s with the correct condition.

In [12]:
def handle_guess(user_guess, random_word, display, guessed): # Here, user_guess is the (valid) guess of the user
    is_guess_correct = True # We set this to True in the beginning and reset it to false later if the user's guess is wrong
    # First, we add the guess to the guessed list
    # Add a line of code to add things to the list
    guessed.append(user_guess)
    # Then, we check if the guess is actually in our word
    # Replace the None here with the appropriate test
    if (user_guess in random_word):
        # We loop over all the letters in the word
        # You have to replace the 0 with the length of the word list
        # i here is used to access the corresponding letter in the guessed and display lists
        for i in range(len(random_word)):
            if user_guess == random_word[i]: # if the i-th letter is equal to our guess
                display[i] = user_guess # then the display list should be updated
                if not (user_guess in guessed): # if the letter is not in the guessed list already...
                    # add it to the list
                    guessed.append(user_guess)
                    # pass # remove this line of code with your code
        print("Your guess is correct!")
    else:
        print("Your guess is wrong.")
        is_guess_correct = False # Wrong guess, so we set the guess back to False
    return is_guess_correct # Notify the user of the function if our guess is correct or not

## Test: Does your Code Work?
Play with the code block below to test your code

Check for a few things:
1. Display contains the letters you supplied, if any;
2. Guess has no duplicate letters;
3. Guesses used should not be increased if your guess is right.

In [13]:
random_word_copy = random_word # copy our random word above

# Reinitialize the word-related variables
random_word = "pedagogical"
guessed = []
display = ['_'] * len(random_word)
letters = list(random_word)

# The actual test
guess_result = handle_guess(get_guess(guessed), random_word, display, guessed)
if not guess_result: # if the guess_result is not right, then
    guesses_used += 1 # add 1 to guesses_used

print(f"guessed = {guessed}, display = {display}, guesses_used = {guesses_used}")

Please enter a letter:  p


Your guess is correct!
guessed = ['p'], display = ['p', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], guesses_used = 0


Run the block below to reset everything.

In [18]:
# Reset the variables to the pre-test state
random_word = random_word_copy
guessed = []
display = ['_'] * len(random_word)
letters = list(random_word)
guesses_used = 0

## Task 4: Putting It All Together
Let's add everything together to complete our Hangman game.

In [21]:
won = False # Indicates whether we have won 
## TODO HERE
# While the guesses_used is less than the limit *and* we have not won yet...
while guesses_used < limit and not won:
    clear_output(wait=True) # clear the screen
    draw(guesses_used)
    print(f"The current revealed letters are: {display}\nYou have guessed the letters: {guessed}")
    print(f"You have {limit - guesses_used} guesses remaining.") # print message for user to know how many guesses they have left 
    user_guess = get_guess(guessed) # Get the valid user guess
    guess_result = handle_guess(user_guess, random_word, display, guessed) # Handle the guess
    if not guess_result: # If the guess is wrong, then...
        guesses_used = guesses_used + 1 # Add 1 to the guesses_used
    draw(guesses_used)
    if display == letters: # if we have revealed every single letter, then
        won = True

clear_output(wait=True)
if not won: # If we exit but have not won, that means we lost
    print(f"You have ran out of guesses. The word is {random_word}. Better luck next time!")
else:
    print(f"The word is {random_word}. Good job!")

   _____ 
  |     |
  |     |
  |     |
  |    
  |    
  |    
__|__     

The current revealed letters are: ['_', '_', '_', '_', '_', '_']
You have guessed the letters: ['p', 's', 'a']
You have 3 guesses remaining.


KeyboardInterrupt: Interrupted by user

## Bonus: Entropy-Based Hangman Solver
This is **very hard**. Please only attempt this if you are confident in your mathematics (specifically probability and logarithms).

## More Math Revision: Exponents and Logarithms

**In general, you can treat logarithms as "black boxes" here. You don't need to understand all of this block if you only treat logarithms as something which takes an input and gives you another number as the output.**

You should have learnt **exponentiation** before. It is basically $a^b$, and we call $a$ the **base** and $b$ the **exponent**.

Exponents are a good way to represent repeated multiplication if the exponent is an integer. Here, we might have to work with non-integer exponents, but they function the same way.

What you may not have learnt before are **logarithms**. Logarithms are the opposite of exponentiation, same as division being the opposite of multiplication.

Given the base, you can recover the exponent of a number.

Logarithms are represented by $log_b(a)$, where $b$ is the base that you want.

An example can be found below:

$2^3 = 8$. Therefore, $log_2(8) = 3$.

## Information: How to Make an Optimal Guess?
Last time, we used binary search in order to cut our problems into half. Since we're only working with a limited range of numbers, that was easy.

However, the situation is much more complex for Hangman. We have to figure out which of the 26 letters to guess, which isn't very easy.

Intuitively, at each point of the game we want to guess the best letter which eliminates the greatest number of possibilities.

We measure the ability of a guess to eliminate possibilities by its **entropy** $H$. The higher the entropy, the more powerful the guess is for reducing the number of possibilities.

$H$ in our case is the sum of all the possible $-p \log_2(p)$, where $p$ is basically the probability of each possible outcome given a word list.

## Example: What is the $H$ of a guess?
This is pretty vague, so let's illustrate this with an example.

Consider a word with three letters (`_`, `_`, `_`), and we have the words `[aca, bcc, fad, ara]`.

Let's guess `a`.

The list of possible outcomes for this guess is then:
- `_`, `_`, `_`;
- `a`, `_`, `_`;
- `_`, `a`, `_`;
- `_`, `_`, `a`;
- `a`, `a`, `_`;
- `_`, `a`, `a`;
- `a`, `_`, `a`; and
- `a`, `a`, `a`.

We call each outcome a *pattern*.

To find the probability of each outcome, we simply find the number of words matching this pattern and divide it by the total number of words.

Then the probabilities of each of these outcomes can be calculated like this:
| Pattern | Number of words matching | Probability $p$ | 
|---|---| --- |
`_`, `_`, `_` | 1 (bcc) | 0.25 (1/4) |
`a`, `_`, `_` | 0 | 0 |
`_`, `a`, `_` | 1 (fad) | 0.25 (1/4) |
`_`, `_`, `a` | 0 | 0 |
`a`, `a`, `_` | 0 | 0 |
`_`, `a`, `a` | 0 | 0 |
`a`, `_`, `a` | 2 (aca, ara) | 0.5 (2/4) |
 `a`, `a`, `a`| 0 | 0 

Then the entropy of guessing `a` is $H = -0.25 \log_2(0.25) + -0.25 \log_2(0.25) + -0.5 \log_2(0.5) = 1.5$.

How do we use this fact?

## Using Entropy to Make Guesses
Now that we know what entropy is, we can use this to make optimal guesses.

1. For each remaining letter:
    1. Calculate the entropy of guessing that letter.
2. Get the letter with maximum entropy and use that as our guess.
3. The guess should give us a new pattern. Remove all words not matching that pattern.

Since this is pretty complex to write on your own, I'll be giving you most of the functions you need.

In [ ]:
import math # We'll be using this a lot
from itertools import product # you don't need to care about this, nor should you

In [ ]:
# Generates all the patterns which can result from a guess.
# Uses _ to represent "not the letter provided", and the letter to represent the letter itself
# for example, the output for the example above would be:
# [[`_`, `_`, `_`], [`a`, `_`, `_`], [`_`, `a`, `_`],
#  [`_`, `_`, `a`], [`a`, `a`, `_`], [`_`, `a`, `a`],
# `[a`, `_`, `a`],  [`a`, `a`, `a`]]
def generate_pattern(letter, word_length):
    return [[*p] for p in product(f'_{letter}', repeat=word_length)] # don't question it

In [ ]:
# Checks if a word matches the letter (pattern and word are both lists).
# pattern are lists like ['_', '_', 'a'], and words are lists like ['a', 'b', 'a']
# the "pattern letter" is equal to the current letter being guessed
def matches(pattern_letter, pattern, word):
    if len(pattern) != len(word): # If the pattern is not of the same length as the word
        return False

    # The loop below counts all the letters which are in the pattern
    letter_in_pattern = [pattern_letter]
    for letter in pattern:
        if letter != '_' and letter not in letter_in_pattern:
            letter_in_pattern.append(letter)
    ## TODO HERE:
    # for each i-th position in the word, do:
    # change the loop to loop from 0 to the length of the word
    for i in range(len(word)):
        if pattern[i] == '_': # if the pattern here is "not pattern_letter provided"...
            if word[i] in letter_in_pattern: # and if and the word contains pattern_letter, then...
                return False
        else: # we know that pattern[i] must be equal to pattern_letter
            if word[i] != pattern[i]: # if the word does not contain pattern_letter at this position, then...
                return False
    return True

In [ ]:
# Generate the entropy of guessing a letter
def calculate_entropy(letter, word_length, word_list):
    total_words = len(word_list) # total words left in the word list
    total_entropy = 0 # total entropy to be calculated
    for pattern in generate_pattern(letter, word_length): # for each generated pattern...
        count = 0 # counts the number of words matching this pattern
        for word in word_list: # for each word in the word list
            if matches(letter, pattern, word): # check if it matches
                count += 1 # add 1 if matches
        p = count / total_words # calculate the p value
        if p != 0.0: # if p is not negative
            total_entropy += - p * math.log(p) / math.log(2) # add to total entropy
    return total_entropy

In [ ]:
# Generates the optimal guess
def get_optimal_guess(guessed_letters, word_length, word_list):
    if len(word_list) == 1:
        for letter in word_list[0]:
            if letter not in guessed_letters:
                return letter
    best_letter = "!"
    best_entropy = -1
    all_letters = "abcdefghijklmnopqrstuvwxyz"
    for letter in all_letters:
        if not (letter in guessed_letters):
            letter_entropy = calculate_entropy(letter, word_length, word_list)
            if letter_entropy >= best_entropy:
                best_entropy = letter_entropy
                best_letter = letter
            print(f"Entropy for {letter}: {letter_entropy}")
    return best_letter

Now that we have all the pieces we need, let's play hangman... optimally.

In [ ]:
random_word = random.choice(word_list) # random.choice chooses something random from the word list
print(f"Your random word is: {random_word}")

In [ ]:
guesses_used = 0 # The number of guesses used
limit = 6 # The maximum number of guesses before you lose
guessed = [] # The guessed number pile; a List
display = ['_'] * len(random_word) # The letters to display; Initially a List of all [_]
letters = list(random_word) # The actual letters of the word; will not be modified

In [ ]:
won = False # Indicates whether we have won 
current_word_list = [p for p in word_list if len(p) == len(random_word)] # the list we use every guessing round
## TODO HERE
# While the guesses_used is less than the limit *and* we have not won yet...
while guesses_used < limit and not won:
    # clear_output(wait=True) # clear the screen
    draw(guesses_used)
    print(f"The current revealed letters are: {display}\nYou have guessed the letters: {guessed}")
    print(f"You have {limit - guesses_used} guesses remaining.") # print message for user to know how many guesses they have left 
    user_guess = get_optimal_guess(guessed, len(random_word), current_word_list) # Get the valid user guess
    print(f"The computer has guessed {user_guess}!")
    guess_result = handle_guess(user_guess, random_word, display, guessed) # Handle the guess
    if not guess_result: # If the guess is wrong, then...
        guesses_used = guesses_used + 1 # Add 1 to the 
    # Filter away all words which don't fit the current pattern
    working_list = current_word_list[:]
    for word in current_word_list:
        if not matches(user_guess, display, word): # if it does not match?
            working_list.remove(word)
    current_word_list = working_list[:]
    print(f"len_word_list = {len(current_word_list)}, wordlist = {current_word_list}")
    draw(guesses_used)
    if display == letters: # if we have revealed every single letter, then
        won = True

# clear_output(wait=True)
if not won: # If we exit but have not won, that means we lost
    print(f"You have ran out of guesses. The word is {random_word}. Better luck next time!")
else:
    print(f"The word is {random_word}. Good job!")